In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.preprocessing import LabelBinarizer,StandardScaler,OrdinalEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from scipy.stats import boxcox
from sklearn.linear_model import LogisticRegression,RidgeClassifier, PassiveAggressiveClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from fancyimpute import KNN, SoftImpute
from imblearn.over_sampling import SMOTE
import joblib
import operator

%matplotlib inline

In [3]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from sklearn.utils import _safe_indexing
sys.modules['sklearn.utils.safe_indexing'] = sklearn.utils._safe_indexing

In [6]:
test = pd.read_csv("LoansTestSet.csv", low_memory=False)

In [16]:
test.describe()

,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,3.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.0
mean,16698.222222,0.333333,7215.555556,2.444444,0.777778,55336.444444,0.444444,781.197778,19.266667,22.333333,12.444444,0.333333,15272.777778,35346.000000,0.333333,0.0
std,6490.181927,0.500000,279.647596,1.589899,0.666667,14038.296354,0.726483,287.208018,6.246599,11.239810,3.678013,0.500000,10441.093522,24396.807768,0.500000,0.0
min,9789.000000,0.000000,6630.000000,0.000000,0.000000,34171.000000,0.000000,274.590000,10.000000,10.000000,9.000000,0.000000,4252.000000,7946.000000,0.000000,0.0
25%,11911.000000,0.000000,7240.000000,1.000000,0.000000,47003.000000,0.000000,590.120000,16.700000,17.500000,10.000000,0.000000,7009.000000,15379.000000,0.000000,0.0
50%,16117.000000,0.000000,7330.000000,3.000000,1.000000,50945.000000,0.000000,871.110000,17.700000,25.000000,12.000000,0.000000,12903.000000,25012.000000,0.000000,0.0
75%,17705.000000,1.000000,7400.000000,3.000000,1.000000,60438.000000,1.000000,892.090000,22.800000,28.500000,13.000000,1.000000,16913.000000,54018.000000,1.000000,0.0
max,28988.000000,1.000000,7420.000000,5.000000,2.000000,81099.000000,2.000000,1244.020000,30.200000,32.000000,21.000000,1.000000,35706.000000,77961.000000,1.000000,0.0


In [7]:
cat_cols = ['Term','Years in current job','Home Ownership','Purpose']

for c in cat_cols:
    test[c] = pd.factorize(test[c])[0]

In [8]:
# Imputing missing data with soft impute
updated_test_data=pd.DataFrame(data=SoftImpute().fit_transform(test[test.columns[3:19]],), columns=test[test.columns[3:19]].columns, index=test.index)


[SoftImpute] Max Singular Value of X_init = 220732.319189
[SoftImpute] Iter 1: observed MAE=393.868714 rank=4
[SoftImpute] Iter 2: observed MAE=393.835770 rank=4
[SoftImpute] Iter 3: observed MAE=393.823069 rank=4
[SoftImpute] Iter 4: observed MAE=393.816892 rank=4
[SoftImpute] Iter 5: observed MAE=393.813120 rank=4
[SoftImpute] Iter 6: observed MAE=393.810394 rank=4
[SoftImpute] Iter 7: observed MAE=393.808205 rank=4
[SoftImpute] Iter 8: observed MAE=393.806333 rank=4
[SoftImpute] Iter 9: observed MAE=393.804668 rank=4
[SoftImpute] Iter 10: observed MAE=393.803152 rank=4
[SoftImpute] Iter 11: observed MAE=393.801748 rank=4
[SoftImpute] Iter 12: observed MAE=393.800437 rank=4
[SoftImpute] Iter 13: observed MAE=393.799203 rank=4
[SoftImpute] Iter 14: observed MAE=393.798037 rank=4
[SoftImpute] Iter 15: observed MAE=393.796931 rank=4
[SoftImpute] Iter 16: observed MAE=393.795881 rank=4
[SoftImpute] Iter 17: observed MAE=393.794882 rank=4
[SoftImpute] Iter 18: observed MAE=393.793931 rank

In [9]:
# Getting the dataset ready pd.get dummies function for dropping the dummy variables
test_data = pd.get_dummies(updated_test_data, drop_first=True)

In [10]:
gbm_pickle = joblib.load('GBM_Model.pkl')

In [11]:
y_pred = gbm_pickle.predict(test_data)


/home/aygul_zagidullina771/.local/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [12]:
y_pred = gbm_pickle.predict_proba(test_data)


/home/aygul_zagidullina771/.local/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [13]:
y_pred_1 = np.where(y_pred == 0, 'Loan Approved', 'Loan Rejected')


In [14]:
test['Loan Status'] = y_pred_1

test.to_csv('Output_Test.csv',index = False)

In [15]:
y_pred_1

array([['Loan Rejected', 'Loan Rejected'],
       ['Loan Rejected', 'Loan Rejected'],
       ['Loan Rejected', 'Loan Rejected'],
       ['Loan Rejected', 'Loan Rejected'],
       ['Loan Rejected', 'Loan Rejected'],
       ['Loan Rejected', 'Loan Rejected'],
       ['Loan Rejected', 'Loan Rejected'],
       ['Loan Rejected', 'Loan Rejected'],
       ['Loan Rejected', 'Loan Rejected']], dtype='<U13')